<a href="https://colab.research.google.com/github/KhaledRobot/ML/blob/main/Court_agent_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `law` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `law`


In [8]:
import pandas as pd
from google.colab import files

# Upload and load CSV (no headers, each row is full case text)
uploaded = files.upload()
cases_df = pd.read_csv(next(iter(uploaded)), header=None, names=['full_case_text'])
print(f"✅ Loaded {len(cases_df)} cases")
print("\nSample case:")
print(cases_df['full_case_text'].iloc[0][:200] + "...")  # Preview first case

In [7]:


# Two–Lawyer Agents (Defense & Prosecution)


from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")           # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


# System prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

PLAINTIFF_SYSTEM = """
You are **Garcia**, *Plaintiff* in this case. Your role:
• Clearly state what the defendant did wrong
• Explain how their actions harmed you
• Specify what compensation/remedy you seek
Style:
• Factual and concise (1-2 sentences per point)
• Avoid emotional language; focus on damages
"""

WITNESS_SYSTEM = """
You are an **eyewitness**. Rules:
• ONLY state what you directly observed
• Never speculate or guess
• If unsure, say "I don't recall"
Example responses:
• "I saw the defendant enter the server room at 11PM"
• "I don't know what they did inside"
"""


# the two agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

# New agents
plaintiff = LawyerAgent("Plaintiff", PLAINTIFF_SYSTEM)
witness = LawyerAgent("Witness", WITNESS_SYSTEM)

# Example dialogue  |  State v. Doe

print("==== Opening statements ====\n")

case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)

# Prosecutor goes first
p_open = prosecution.respond(
    f"Opening statement to the Court. Background: {case_background}"
)
print("PROSECUTOR:", p_open, "\n")

# Defense responds
d_open = defense.respond(
    "Opening statement to the Court responding to the prosecution."
)
print("DEFENSE   :", d_open, "\n")

# Prosecutor rebuttal
p_rebut = prosecution.respond(
    "Brief rebuttal to the defense's opening."
)
print("PROSECUTOR:", p_rebut, "\n")

# ====== NEW ADDITIONS ======
print("\n==== Plaintiff's Testimony ====")  # NEW
plaint = plaintiff.respond(
    f"Describe your damages to the Court. Background: {case_background}"
)
print("PLAINTIFF:", plaint, "\n")  # NEW

print("==== Witness Examination ====")  # NEW
# Prosecutor questions witness
p_question = prosecution.respond(
    "Ask the witness one specific question about the server logs."
)
print("PROSECUTOR to Witness:", p_question)  # NEW
w_answer = witness.respond(p_question)  # NEW
print("WITNESS:", w_answer, "\n")  # NEW

# Defense cross-examination  # NEW
d_question = defense.respond(
    "Ask the witness one follow-up question to challenge their testimony."
)
print("DEFENSE to Witness:", d_question)  # NEW
w_answer2 = witness.respond(d_question)  # NEW
print("WITNESS:", w_answer2, "\n")  # NEW

# Final arguments referencing new testimony  # NEW
print("==== Closing Arguments ====")  # NEW
p_close = prosecution.respond(  # NEW
    f"Closing statement incorporating witness testimony: '{w_answer}' and plaintiff statement: '{plaint}'"
)
print("PROSECUTOR:", p_close, "\n")  # NEW

d_close = defense.respond(  # NEW
    f"Final argument addressing witness testimony: '{w_answer2}'"
)
print("DEFENSE:", d_close)  # NEW


==== Opening statements ====

PROSECUTOR: "Ladies and Gentlemen of the jury, today, you will be presented with clear and convincing evidence that John Doe, my client's former employer, a company dedicated to innovation and hard work, has been the victim of a serious infringement of their rights. This crime is not just about stolen intellectual property; it's about the fundamental principles of fairness and respect for another's hard work that underpin our society.


John Doe, a skilled software engineer, played a critical role in developing proprietary algorithms essential to the success of my client's business. These algorithms are not only the result of countless hours of dedication but also a form of intellectual property protected under the law. On the day before John Doe left his job, there were large downloads from the company's servers, indicating he may have accessed confidential information.


My client, a pioneer in its field, has invested heavily in research and development.